<div class="alert alert-block alert-info">
<b>

## Homework III: Option Pricing ##
### Subject: Python for Finance and Optimization ###
### Homework Submission By: GILL Sarbjit & RATHOD Salonibah ### 
</b>
</div>

<div class="alert alert-block alert-warning"> 
<b> Task 1 of 4:
    Propose and code an implicit scheme for a call-spread – long leg: at-the-money call with 3-year maturity; short leg: call of maturity equal to 3 years with moneyness 150%.

</div>

In [1]:
import numpy as np

Our "Call Spread" Pricing Approach:
We create the grid with vertical axis being asset prices (S), while the horizontal axis will resemble the time (T). We use two methods of finite differences to approximate the PDE. Implicit scheme doesn't depend on quantities from previous state whereas the Crank-Nicholson method is an implicit method that is weighted between the explicit method and the implicit method hence we write the code that uses boundary conditions:
<ul>
    <li> upper boundary whose value is derived by taking the expectation of E[Payoff of long call + Payoff of short call] where price is the initial asset price compounded at the riskless rate multiplied by the terminal time value: max(S-K1,0)+ - max(S-K2,0)+. (here K is the strike or exercise price) </li>
    <li> lower boundary is where the option price is 0. </li>
    <li> The time at expiration, t = T, and the option can be calculated for all the different stock prices for this boundary condition.</li>
Option prices will be calculated for each time step backwards from t = T.
</ul>

In [2]:
def pde_implicit_scheme_for_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t):
# first, we build the grid
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t]) # defining price 
    g = lambda S : (np.maximum(S-K1,0.))-(np.maximum(S-K2,0.)) # combining the payoff of long and short calls
    p[:,0] = g(np.exp(xs))
    p[0,:] = 0. # lower boundary
    p[-1,:] = (K2-K1) * np.exp(-r*ts) # upper boundary
    # now we define code to solve the implicit scheme through finite differences
    d = 1.+dt*(r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2) # dominant diagonal
    sup_d = -dt*((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2) # alpha
    inf_d = -dt*(0.5 * sigma**2 / dx**2) # gamma
    A = np.diag(d * np.ones(nb_x-2)) + np.diag(sup_d * np.ones(nb_x-3), 1) + np.diag(inf_d * np.ones(nb_x-3), -1) # defining matrix A
    v = np.zeros_like(p[1:-1,0])
    v[-1] = 1.
    invA = np.linalg.inv(A)
    for t in range(1,nb_t):
        p[1:-1,t] = invA @ (p[1:-1,t-1] - sup_d * p[-1,t] * v)   
    return p, p[nb_x_side,-1]

In [3]:
S_0 = 10. # intial price
H = np.log(9*S_0) - np.log(S_0) # H being sufficiently large
nb_x_side = 1000 # defining points on the grid
nb_t = 300 # defining points on the grid
S_min = S_0 * np.exp(-H)
S_max = S_0 * np.exp(H)
K1 = 10. # long call at the money
K2 = 2*S_0/3 # short call at moneyness 150% being in the money
r = 0.02 # riskless rate
sigma = 0.25 # volatility
T = 3. # time to maturity
S_min, S_max

(1.1111111111111114, 89.99999999999999)

In [4]:
%%time
table_call_spread_price_implicit, call_spread_price_implicit = pde_implicit_scheme_for_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t)
call_spread_price_implicit

print(f"The price of the call spread under the implicit scheme is: {call_spread_price_implicit}")

The price of the call spread under the implicit scheme is: -1.9948243179193605
CPU times: user 2.21 s, sys: 125 ms, total: 2.34 s
Wall time: 758 ms


<div class="alert alert-block alert-warning"> 
<b> Task 2 of 4:
    Propose and code a Crank-Nicolson for a call-spread – long leg: at-the-money call with 3-year maturity ; short leg: call of maturity equal to 3 years with moneyness 150%.

</div>

In [5]:
def crank_scheme_for_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    g = lambda S : (np.maximum(S-K1,0.))-(np.maximum(S-K2,0.))
    p[:,0] = g(np.exp(xs))
    p[0,:] = 0.
    p[-1,:] = (K2-K1) * np.exp(-r*ts)
    # now we define Crank-Nicholson code to solve the scheme through finite differences taking aspects of both implicit and explicit
    d = 0.5 * dt * (r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -0.5 * dt * ((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -0.5 * dt * (0.5 * sigma**2 / dx**2)
    A = (np.diag((1.+d)*np.ones(nb_x))) + (sup_d * np.diag(np.ones(nb_x -1),1)) + (inf_d * np.diag(np.ones(nb_x -1),-1))
    B = (np.diag((1.-d)*np.ones(nb_x))) - (sup_d * np.diag(np.ones(nb_x -1),1)) - (inf_d * np.diag(np.ones(nb_x -1),-1))
    C = np.linalg.inv(A) @ B
    for t in range(1,nb_t):
        p[1:-1,t] = (C @ p[:,t-1])[1:-1]
    return p, p[nb_x_side,-1]

In [6]:
%%time

table_call_spread_crank, call_spread_price_crank = crank_scheme_for_call_spread(S_0, r, sigma, T, K1, K2, H, nb_x_side, nb_t)
call_spread_price_crank

print(f"The price of the call spread under the Crank Nicholson scheme is: {call_spread_price_crank}")

The price of the call spread under the Crank Nicholson scheme is: -1.994200022934274
CPU times: user 3.42 s, sys: 165 ms, total: 3.59 s
Wall time: 959 ms


In [7]:
# Exploring the consistency between the two prices for the same call-spread option

price_diff_callspread = np.abs(call_spread_price_crank - call_spread_price_implicit)
print(f"The absolute price difference between the two call spread prices is: {price_diff_callspread}")

The absolute price difference between the two call spread prices is: 0.0006242949850865376


<div class="alert alert-block alert-warning"> 
<b> Task 3 of 4:
    Propose and code an implicit scheme for a 10-year maturity put option with moneyness 80%.

</div>

**Please Note: We are pricing a long put below.**

Our "Put Option" Pricing Approach:
We create the grid with vertical axis being asset prices (S), while the horizontal axis will resemble the time (T). We use two methods of finite differences to approximate the PDE. 
Implicit scheme doesn't depend on quantities from previous state whereas the Crank-Nicholson method is an implicit method that is weighted between the explicit method and the implicit method hence we write the code that uses boundary conditions:
<ul>
    <li> upper boundary is 0. </li>
    <li> lower boundary's value is derived by taking the expectation of E[Payoff of long put] where price is the initial asset price compounded at the riskless rate multiplied by the terminal time value: max(K-S,0)+ (here K is the strike or exercise price) </li> the option values for maximal asset value is Smax hence max(S-K,0). (here K is the strike or exercise price) </li>
    <li> The time at expiration, t = T, and the option can be calculated for all the different stock prices for this boundary condition.</li>
Option prices will be calculated for each time step backwards from t = T.
</ul>   

In [8]:
def pde_implicit_scheme_for_put(S_0, r, sigma, T, K, H, nb_x_side, nb_t):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    g = lambda S : np.maximum(0.,K-S) # payoff function for put
    p[:,0] = g(np.exp(xs))
    p[0,:] = K * np.exp(-r*ts) - (np.exp(np.log(S_0)-H))
    p[-1,:] = 0.
    d = 1.+dt*(r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -dt*((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -dt*(0.5 * sigma**2 / dx**2)
    A = np.diag(d * np.ones(nb_x-2)) + np.diag(sup_d * np.ones(nb_x-3), 1) + np.diag(inf_d * np.ones(nb_x-3), -1)
    v = np.zeros_like(p[1:-1,0])
    v[-1] = 0. #last element is no longer 1
    invA = np.linalg.inv(A)
    for t in range(1,nb_t):
        p[1:-1,t] = invA @ (p[1:-1,t-1] - inf_d * p[0,t] * v)   
    return p, p[nb_x_side,-1]

In [9]:
S_0 = 10.
H = np.log(30*S_0) - np.log(S_0) # due to increase in maturity tenure we further increased H
nb_x_side = 1000
nb_t = 300
S_min = S_0 * np.exp(-H)
S_max = S_0 * np.exp(H)
K = 0.8*S_0 # moneyness 80% at out of money
r = 0.02
sigma = 0.25
T = 10.
S_min, S_max

(0.3333333333333335, 299.9999999999999)

In [10]:
%%time
table_put_price_implicit, put_price_implicit = pde_implicit_scheme_for_put(S_0, r, sigma, T, K, H, nb_x_side, nb_t)
put_price_implicit

print(f"The price of the long put option under the implicit scheme is: {put_price_implicit}")

The price of the long put option under the implicit scheme is: 1.1482707813585011
CPU times: user 2.41 s, sys: 54.5 ms, total: 2.46 s
Wall time: 637 ms


<div class="alert alert-block alert-warning"> 
<b> Task 4 of 4:
    Propose and code a Crank-Nicolson scheme for a 10-year maturity put option with moneyness 80%.

</div>

In [11]:
def crank_scheme_for_put(S_0, r, sigma, T, K, H, nb_x_side, nb_t):
    nb_x = 2 * nb_x_side + 1
    xs = np.linspace(np.log(S_0)-H, np.log(S_0)+H, nb_x)
    dx = 2. * H / (nb_x - 1.) 
    dt = T / (nb_t - 1.)
    ts = np.linspace(0, T, nb_t)
    p = np.empty([nb_x, nb_t])
    g = lambda S : np.maximum(0.,K-S)
    p[:,0] = g(np.exp(xs))
    p[0,:] = K * np.exp(-r*ts) - (np.exp(np.log(S_0)-H))
    p[-1,:] = 0.
    d = 0.5 * dt * (r + (r-0.5*sigma**2)/dx + sigma**2 / dx**2)
    sup_d = -0.5 * dt * ((r-0.5*sigma**2)/dx + 0.5 * sigma**2 / dx**2)
    inf_d = -0.5 * dt * (0.5 * sigma**2 / dx**2)
    A = (np.diag((1.+d)*np.ones(nb_x))) + (sup_d * np.diag(np.ones(nb_x -1),1)) + (inf_d * np.diag(np.ones(nb_x -1),-1))
    B = (np.diag((1.-d)*np.ones(nb_x))) - (sup_d * np.diag(np.ones(nb_x -1),1)) - (inf_d * np.diag(np.ones(nb_x -1),-1))
    C = np.linalg.inv(A) @ B
    for t in range(1,nb_t):
        p[1:-1,t] = (C @ p[:,t-1])[1:-1]
    return p, p[nb_x_side,-1]

In [12]:
%%time
table_put_price_crank, put_price_crank = crank_scheme_for_put(S_0, r, sigma, T, K, H, nb_x_side, nb_t)
put_price_crank

print(f"The price of the long put under the Crank Nicholson scheme is: {put_price_crank}")

The price of the long put under the Crank Nicholson scheme is: 1.149437414631536
CPU times: user 3.48 s, sys: 91.9 ms, total: 3.57 s
Wall time: 964 ms


In [13]:
# Exploring the consistency between the two prices for the same long put option

price_diff_callspread2 = np.abs(put_price_implicit - put_price_crank)
print(f"The absolute price difference between the two call spread prices is: {price_diff_callspread2}")

The absolute price difference between the two call spread prices is: 0.0011666332730349005


<div class="alert alert-block alert-info">
<b>

    Thank You!
</b>
</div>